In [ ]:
from pathlib import Path

from cool_styles import coastalarvest

from matplotlib import pyplot as plt

from thesis_breath_frequency.project_configs import ProjectPaths

project_paths = ProjectPaths()

plt.style.use(coastalarvest)

source_excel_path = project_paths.data_folder.raw.joinpath("Respiratorio CPET BR.xlsx")